## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,Weather_Database,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,0,Moerai,-22.43,-151.33,79.99,77,7,12.97,PF,2021-01-02 01:21:23,clear sky
1,1,Ponta Do Sol,32.67,-17.10,59.00,67,75,6.93,PT,2021-01-02 01:15:21,broken clouds
2,2,Georgetown,5.41,100.34,78.80,83,20,6.29,MY,2021-01-02 01:14:31,moderate rain
3,3,Nikolskoye,59.70,30.79,33.01,98,90,6.71,RU,2021-01-02 01:15:12,light shower snow
4,4,Punta Arenas,-53.15,-70.92,60.80,63,0,21.92,CL,2021-01-02 01:21:23,clear sky


In [3]:
# Prompt the user to enter minimum and maximum temperature criteria 
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Weather_Database,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,0,Moerai,-22.43,-151.33,79.99,77,7,12.97,PF,2021-01-02 01:21:23,clear sky
2,2,Georgetown,5.41,100.34,78.80,83,20,6.29,MY,2021-01-02 01:14:31,moderate rain
9,9,Hilo,19.73,-155.09,77.00,73,40,6.93,US,2021-01-02 01:17:22,light rain
16,16,Kahului,20.89,-156.47,82.40,48,40,23.04,US,2021-01-02 01:16:11,scattered clouds
24,24,Hithadhoo,-0.60,73.08,81.84,73,1,7.00,MV,2021-01-02 01:10:15,moderate rain
30,30,Kapaa,22.08,-159.32,78.80,74,40,16.11,US,2021-01-02 01:15:12,scattered clouds
31,31,Urucara,-2.54,-57.76,75.79,91,100,2.26,BR,2021-01-02 01:21:31,overcast clouds
35,35,Atuona,-9.80,-139.03,78.21,75,2,15.30,PF,2021-01-02 01:15:36,clear sky
38,38,Alyangula,-13.85,136.42,78.80,100,75,11.41,AU,2021-01-02 01:21:33,broken clouds
39,39,Tual,-5.67,132.75,84.63,73,100,12.46,ID,2021-01-02 01:17:52,light rain


In [5]:
# Determine if there are any empty rows.
preferred_cities_df.count()

Weather_Database       179
City                   179
Lat                    179
Lng                    179
Max Temp               179
Humidity               179
Cloudiness             179
Wind Speed             179
Country                179
Date                   179
Current Description    179
dtype: int64

In [6]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description","Lat", "Lng"]].copy()

# Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Moerai,PF,79.99,clear sky,-22.43,-151.33,
2,Georgetown,MY,78.80,moderate rain,5.41,100.34,
9,Hilo,US,77.00,light rain,19.73,-155.09,
16,Kahului,US,82.40,scattered clouds,20.89,-156.47,
24,Hithadhoo,MV,81.84,moderate rain,-0.60,73.08,
30,Kapaa,US,78.80,scattered clouds,22.08,-159.32,
31,Urucara,BR,75.79,overcast clouds,-2.54,-57.76,
35,Atuona,PF,78.21,clear sky,-9.80,-139.03,
38,Alyangula,AU,78.80,broken clouds,-13.85,136.42,
39,Tual,ID,84.63,light rain,-5.67,132.75,


In [7]:
# Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
  
    # Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json() 
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city.    
hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
# Grab the first hotel from the results and store the name.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")

In [8]:
# Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_Vacation.csv"
# Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [9]:
# Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]

In [10]:
# Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Display the figure
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

In [11]:
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))